In [1]:
import numpy as np
import scipy.io
import itertools # for generting all possible n-grams
import sklearn.metrics
import os 
#for visualization
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

n_g=2 # n-gram size
data = scipy.io.loadmat('languages_data.mat')
alphabet=(data['alphabet'])
alphabet=str(alphabet[0]) # turn into string
N_GRAMS=[p for p in itertools.product(alphabet, repeat=n_g)] # get all possible n-grams

# get data from the file
chop_size=int(data['chop_size'])
exm_size=int(data['exm_size'])
langLabels=(data['langLabels'])
testing=(data['testing'])
training=(data['training'])




C:\Users\sexysehe\AppData\Local\Temp\ipykernel_16460\2071213570.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  chop_size=int(data['chop_size'])
C:\Users\sexysehe\AppData\Local\Temp\ipykernel_16460\2071213570.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  exm_size=int(data['exm_size'])


## Process files from 21 languages

In [22]:
import string
import unicodedata
import re
#create a function to preprocess the sentences
def preprocess_sentence(sentence):
    # Lowercasing
    sentence = sentence.lower()

    # Remove Numbers
    sentence = ''.join(char for char in sentence if not char.isdigit())

    # Remove Punctuation
    sentence = ''.join(char for char in sentence if char not in string.punctuation)

    #Remove special characters
    sentence = ''.join(char for char in sentence if char.isalnum() or char.isspace())

    #Some characters are not considered digits nor letters such as ¼, so we use another condition
    unwanted_characters = ['¼','½','¾']
    sentence = ''.join(char for char in sentence if char not in  unwanted_characters )


    return sentence


#folder path with all the files
from pathlib import Path
parent_dir_sentences = "./sentences/"
own_training_data = {}
own_training_data["labels"] = []
own_training_data["sentences"] = [""]*21

for (i,path) in enumerate(os.listdir(parent_dir_sentences)):
    #for good measure we check if what we are iterating is a file (we know all of them are but this is good measure)
    if os.path.isfile(os.path.join(parent_dir_sentences, path)):
        own_training_data["labels"].append((Path(path).stem))
        with open(os.path.join(parent_dir_sentences, path), "r", encoding="utf-8") as my_file_read:
            for line in my_file_read:
                index, content = line.strip().split('\t')
                if int(index) >= 100:
                    continue
                sentence = preprocess_sentence(content)
                own_training_data["sentences"][i] += sentence
    



In [29]:
print(own_training_data["sentences"][0])


 пъти позле е към момента кирил петков сравнен с бойко борисов  по кадастралната карта и кадастралните регистри на с изгрев общ ти си тъпак мм е големината на нимфата като се нахрани става  пъти поголяма за комунистите които изкрадоха цяла българия и кусур милиона за тия куци коне ли ве  лв е приблизителната стойност на средното ниво на прага за регистрация в ес и ек няма да има основание да откаже аргументира се мартин димитров дипломи е семейната шега относно навика на величка да се връща към учебниците години покъсно будителството продължава да е силата с която просветителите разпалват искрата за знание във всеки от нас изтъкна премиерът донев грама мир са много повече от тонове оръжия по време на война добавя мамалев цитирайки еврейска поговорка дни безпрецедентен комплекс от кризи и множество предприети от държавата действия компенсация за бизнеса е безсмисленапроцентовата подкрепа на българската държава към украйна е факт пъти да имах възможност да върна времето назад  пъти щях д

In [ ]:
# Print an example of a text in the training data
ind=3;
print(langLabels[0][ind])
print(training[0][ind])
print (chop_size)

In [ ]:
# collect n-gram statitics for all training data 
TR_grams=np.zeros((len(N_GRAMS),langLabels.size), dtype='float') # initialize n-gram statistics for the  training

for i in range(langLabels.size):
    print(langLabels[0][i])

    buf_chop=training[0][i] #pick the current portion
    buf_chop=np.array2string(buf_chop) # get to string    
    buf_chop=(buf_chop[3:len(buf_chop)-2]) # get rid of artefacts like []
    
    for jj in range(len(buf_chop)-(n_g-1)):        
        ngc=buf_chop [jj:(jj+(n_g))] #pick current n-gram
        ngc1= tuple(ngc)
        ind_ngc=N_GRAMS.index(ngc1) # find index in  N_GRAMS
        TR_grams[ind_ngc,i]+=1 #increment the corresponding statisticss

    TR_grams[:,i]=TR_grams[:,i]/np.linalg.norm(TR_grams[:,i]) # normalize

print(TR_grams[:,0])    

In [ ]:
# collect n-gram statitics for all testing data 


GT = [None] * langLabels.size * exm_size   #ground truth  
PR = [None] * langLabels.size * exm_size   #predicton

for i in range(langLabels.size):    
    for j in range(exm_size): 
        TS_gram=np.zeros((len(N_GRAMS),1), dtype='float') # initialize n-gram statistics for the  test example
        buf_chop=testing[j][i] #pick the current portion
        buf_chop=np.array2string(buf_chop) # get to string    
        buf_chop=buf_chop[3:len(buf_chop)-2] # get rid of artefacts like []
        
        for jj in range(len(buf_chop)-(n_g-1)): 
            ngc=buf_chop [jj:(jj+(n_g))] #pick current n-gram
            ngc= tuple(ngc)
            ind_ngc=N_GRAMS.index(ngc) # find index in  N_GRAMS            
            TS_gram[ind_ngc,0]+=1 #increment the corresponding statisticss
            
        DP=np.dot(TS_gram.transpose(),TR_grams)
        ind=np.argmax(DP) #  index of predictd language
        PR[ i*exm_size +j ]=np.array2string(langLabels[0][ind])
        GT[ i*exm_size +j ]=np.array2string(langLabels[0][i]) # add ground truth

In [ ]:
# accuracy 
acc=0.0
for i in range(len(GT)):
   acc+=(PR[i]==GT[i])

acc=acc/len(GT) 
print(acc)

In [ ]:
langLabels[0][1][0].tolist()
Labels_arr=np.empty(21).astype(str)
for i in range(21):
    Labels_arr[i]=langLabels[0][i][0]
    
Labels_arr


In [ ]:
conf_mat= sklearn.metrics.confusion_matrix(GT, PR, labels=None, sample_weight=None)

#plot confusion matrix
%matplotlib inline
df_cm = pd.DataFrame(conf_mat, index = [i for i in Labels_arr], columns = [j for j in Labels_arr])
plt.figure(figsize = (18,18))
sn.heatmap(df_cm, annot=True)